# Introduction
A notebook to deal with Training and Testing Analysis of S-ORCT: regularized version of ORCT. In this notebook the function which express a global regularization term is taken from _"Sparsity in Optimal Randomized Classification Trees"_ (Blanquero et Al. 2018)

### Remark
* We use data from Iris dataset: for a generalized version to manage any kind of datasets look at notebook 'Analysis with Class'

In [ ]:
# dataframe management
import pandas as pd
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Preprocessing of dataset
Let's load the Iris dataset:

In [2]:
iris = pd.read_csv('... .csv') #IrisCategorical.csv
iris = iris.drop('Id', axis=1)
iris_std = iris.copy()
iris.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
scaler = MinMaxScaler() # also MaxAbsScaler()

In [4]:
#Preprocessing: we get the columns names of features which have to be standardized
columns_names = list(iris)
index_features = list(range(0,len(iris_std.columns)-1))

#The name of the classes K
classes = iris_std['Species'].unique().tolist()
classes_en = [i for i in range(len(classes))] 

#Encoder processing
le = preprocessing.LabelEncoder()
le.fit(iris_std['Species'])

iris_std['Species'] = le.transform(iris_std['Species']) 

#Scaling phase
iris_std[columns_names[0:4]] = scaler.fit_transform(iris_std[columns_names[0:4]])

iris_std.head(1)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,0.222222,0.625,0.067797,0.041667,0


Splitting the dataset between __Training & Testing Sets__

In [5]:
df = iris_std[columns_names[:-1]]
y = iris_std[columns_names[4]]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25)
df_train = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])

Objects useful to deal with trees (of depth 2) and their topology

In [6]:
BF_in_NL_R = {4:[],5:[2],6:[1],7:[1,3]}
BF_in_NL_L = {4:[1,2],5:[1],6:[3],7:[]}
I_in_k = {i : list(df_train[df_train['Species']== i].index) for i in range(len(classes))}
my_W = {(i,j): 0.5 if i != j else 0 for i in classes_en for j in classes_en}
index_instances = list(X_train.index)
my_x = {(i,j): df_train.loc[i][j] for i in index_instances for j in index_features}

In [7]:
def B_in_NR(model, i):
    if i==4:
        return []
    elif i==5:
        return [2]
    elif i==6:
        return [1]
    elif i==7:
        return [1,3]
def B_in_NL(model, i):
    if i==4:
        return [1,2]
    elif i==5:
        return [1]
    elif i==6:
        return [3]
    elif i==7:
        return []

def I_k(model,i):
    if i==0:
        return I_in_k[0]
    elif i==1:
        return I_in_k[1]
    elif i==2:
        return I_in_k[2]

# Model definition
We initialize the __model__ and the sets K, N_L, N_B, I, I_k, N_L_L, N_L_R and f_s are declared abstractly using the Set component:

In [14]:
model = ConcreteModel() #ConcretModel()
# Instances & Classes
# Assume a dict I_in_k, with keys k and values of a list of I's in that k

model.I = Set(initialize=set(i for k in I_in_k for i in I_in_k[k]))
model.K = Set(initialize=I_in_k.keys())
model.I_k = Set(model.K,initialize=I_k)    ##########################

# Features
model.f_s =Set(initialize=index_features)

# Nodes Leaf N_L & Nodes Breanch N_B
model.N_B = Set(initialize=set(i for k in BF_in_NL_R for i in BF_in_NL_R[k]))
model.N_L = Set(initialize=BF_in_NL_R.keys())
model.N_L_R = Set(model.N_L,initialize=B_in_NR)
model.N_L_L = Set(model.N_L,initialize=B_in_NL)

Similarly, the model parameters are defined abstractly using the __Param__ component:

In [15]:
# Cost of misclassification
model.W = Param(model.K, model.K, within=NonNegativeReals, initialize=my_W)

# Value for the instance i-th of the feature j-th
model.x = Param(model.I, model.f_s, within=PercentFraction, initialize=my_x)

# Value for the lambda of global generalization
model.lam_glob = Param(initialize=2)

The __Var__ component is used to define the decision variables:

In [16]:
#random initialization
init_a = np.random.uniform(low=-1.0, high=1.0, size=None)
init_beta = np.random.uniform(low=0.0, high=1.0, size=None)
init_mu = np.random.uniform(low=-1.0, high=1.0, size=None)
init_C = np.random.uniform(low=0.0, high=1.0, size=None)
init_P = np.random.uniform(low=0.0, high=1.0, size=None)
init_p = np.random.uniform(low=0.0, high=1.0, size=None)

# The weigths of feature j-th in breanch node t-th
model.a = Var(model.f_s, model.N_B, within=Reals, bounds = (-1.0,1.0),initialize=init_a)

#auxiliary variables for smooth version of global regularization
model.beta = Var(model.f_s, within= PercentFraction, initialize=init_beta)
# The intercepts of the linear combinations correspond to decision variables
model.mu = Var(model.N_B, within = Reals, bounds = (-1.0,1.0),initialize=init_mu)

# The variables thtat take into account if node t is labeled with class k
model.C = Var(model.K, model.N_L, within = PercentFraction,initialize=init_C)

# An auxiliary variables
model.P = Var(model.I,model.N_L,within = PercentFraction,initialize=init_P)
model.p = Var(model.I,model.N_B,within = PercentFraction,initialize=init_p)

Several definition of functions: tools useful to characterize the objective function

In [17]:
# Minimize the cost of misclassification
def cost_rule(model):
    return sum( sum( sum( model.P[i,t]* sum(model.W[k,j]*model.C[j,t] for j in model.K if k!=j)  for t in model.N_L) for i in model.I_k[k] ) for k in model.K ) + model.lam_glob*sum(model.beta[j] for j in model.f_s)
model.cost = Objective(rule=cost_rule, sense=minimize)

In [18]:
# We must add the following set of constraints for making a single class prediction at each leaf node:
def Pr(model,i,tl):
    return  reduce(operator.mul,(model.p[i,t] for t in model.N_L_L[tl]),1)*reduce(operator.mul,(1-model.p[i,tr] for tr in model.N_L_R[tl]),1) == model.P[i,tl]
model.Pr = Constraint(model.I,model.N_L, rule=Pr)

def pr(model, i , tb):
    return 1 / (1 + exp(-512*(   (sum(model.x[i,j]*model.a[j,tb]for j in model.f_s)/4)-model.mu[tb]  ))) ==model.p[i,tb]
model.pr = Constraint(model.I,model.N_B, rule=pr)

Similarly, rule functions are used to define constraint expressions in the __Constraint__ component:

In [19]:
# We must add the following set of constraints for making a single class prediction at each leaf node:
def class_in_leaf(model, tl):
    return  sum(model.C[k,tl] for k in model.K) == 1
model.class_in_leaf = Constraint(model.N_L, rule=class_in_leaf)

# We force each class k to be identified by, at least, one terminal node, by adding the set of constraints below:
def leaf_in_class(model,k):
    return sum(model.C[k,tl] for tl in model.N_L) >=1
model.leaf_in_class = Constraint(model.K, rule=leaf_in_class)

In [20]:
#The following set of constraints uanbles to manage global regularization
def global_min(model,f,tb):
    return model.beta[f]>= model.a[f,tb]
model.globalmin = Constraint(model.f_s, model.N_B, rule=global_min)

def global_ma(model,f,tb):
    return model.beta[f]>= -model.a[f,tb]
model.globalma = Constraint(model.f_s, model.N_B, rule=global_ma)

In [21]:
opt = SolverFactory('ipopt',executable='C:/.../ipopt.exe')# in executable the directory path of ipopt.exe
# Create a model instance and optimize
#instance = model.create_instance()
results = opt.solve(model,tee=True)
#instance.display()

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:     3357
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:     1165

Total number of variables............................:      815
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      815


  72 3.6772531e+001 4.02e-002 2.79e+002  -1.0 2.44e-001    -  7.23e-001 1.00e+000f  1
  73 3.7121661e+001 1.03e-002 1.41e+001  -1.0 1.59e-001    -  1.00e+000 1.00e+000h  1
  74 3.7122808e+001 1.91e-006 1.67e-001  -1.0 1.69e-002   1.0 1.00e+000 1.00e+000h  1
  75 3.5361663e+001 1.10e-003 9.52e-001  -2.5 1.08e-001   0.5 9.00e-001 8.71e-001f  1
  76 3.4504140e+001 3.43e-004 4.92e-001  -2.5 5.64e-002   0.9 1.00e+000 9.52e-001f  1
  77 3.3422754e+001 6.73e-004 1.12e+000  -2.5 1.79e-001   0.5 1.00e+000 3.66e-001f  1
  78 3.2417271e+001 5.32e-004 4.01e-001  -2.5 5.14e-002   0.9 1.00e+000 1.00e+000f  1
  79 3.2081117e+001 5.50e-004 1.19e+000  -2.5 1.83e-001   0.4 1.00e+000 1.05e-001f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 3.0963862e+001 8.02e-004 1.16e+000  -2.5 3.94e-002   0.8 1.00e+000 1.00e+000f  1
  81 2.7501523e+001 7.59e-003 2.51e+001  -2.5 1.67e-001   0.4 1.00e+000 9.76e-001f  1
  82 2.6826369e+001 3.88e-003 1.31e+001  -2.5 7.81e-002   0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160 2.8775628e+000 3.13e-003 8.56e+003  -3.8 3.89e-002    -  1.00e+000 4.16e-001h  1
 161 2.8655575e+000 2.25e-003 2.55e+007  -3.8 3.58e-002    -  2.01e-003 1.00e+000f  1
 162 2.8622462e+000 1.58e-003 6.00e+004  -3.8 2.70e-002    -  4.83e-001 1.00e+000h  1
 163 2.8672444e+000 6.08e-004 4.87e+004  -3.8 1.68e-002    -  2.17e-001 1.00e+000h  1
 164 2.8695819e+000 3.78e-005 2.03e-001  -3.8 5.74e-003    -  1.00e+000 1.00e+000h  1
 165 2.8699541e+000 2.77e-006 2.32e-003  -3.8 1.31e-003    -  1.00e+000 1.00e+000h  1
 166 2.8493508e+000 6.54e-004 8.85e+003  -5.7 4.00e-002    -  8.82e-001 2.86e-001f  1
 167 2.8337778e+000 4.92e-004 5.03e+003  -5.7 3.36e-002    -  9.70e-001 5.25e-001h  1
 168 2.8334513e+000 4.80e-004 4.91e+003  -5.7 1.96e-002    -  4.22e-002 2.48e-002h  1
 169 2.8303242e+000 4.00e-004 3.08e+004  -5.7 2.04e-002    -  7.83e-002 2.49e-001f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) 

Complementarity.........:  2.5148906455900119e-009   2.5148906455900119e-009
Overall NLP error.......:  5.7455738534294822e-009   5.7455738534294822e-009


Number of objective function evaluations             = 292
Number of objective gradient evaluations             = 222
Number of equality constraint evaluations            = 292
Number of inequality constraint evaluations          = 292
Number of equality constraint Jacobian evaluations   = 244
Number of inequality constraint Jacobian evaluations = 244
Number of Lagrangian Hessian evaluations             = 235
Total CPU secs in IPOPT (w/o function evaluations)   =      2.611
Total CPU secs in NLP function evaluations           =      0.243

EXIT: Optimal Solution Found.


In [67]:
print(results)
print(value(model.cost))


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 791
  Number of variables: 811
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 5.3325982093811035
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

1.1558196218933843


Several definition of functions: tools useful to deal with __Test Analysis__

In [26]:
# Function to store the variables results
def extraction_va(model):
    
    mu = {str(model.mu[i]): model.mu[i].value for i in model.mu}
    a = {str(model.a[i]): model.a[i].value for i in model.a}
    C = {str(model.C[i]): model.C[i].value for i in model.C}
    beta = {str(model.beta[i]): model.beta[i].value for i in model.beta}
    
    return {'mu': mu,'a':a ,'C':C, 'beta':beta}

In [27]:
def my_sigmoid(a,x,mu,scale=512):
    l = len(x)
    val = (sum([a[i]*x   for i, x in enumerate(x)]) / l) - mu 
    # The default value is 512 as suggested in Blanquero et Al.
    return 1 / (1 + math.exp(-scale*val))

# An easy way to manage product within elements of an iterable object
def multiply_numpy(iterable):
    return np.prod(np.array(iterable))

# Calculate the probability of an individual falling into a given leaf node:
def Prob(model,var,x, leaf_idx):
    left = [my_sigmoid(list(var['a']['a['+str(i)+','+str(tl)+']'] for i in index_features),x,var['mu']['mu['+str(tl)+']']) for tl in model.N_L_L[leaf_idx] ]
    right = [1-my_sigmoid(list(var['a']['a['+str(i)+','+str(tr)+']'] for i in index_features),x,var['mu']['mu['+str(tr)+']']) for tr in model.N_L_R[leaf_idx] ]
    return multiply_numpy(left)*multiply_numpy(right)

#Calculate the predicted label of a single instance
def comp_label(model,x,var):
    prob ={k : sum(Prob(model,var,x,i)*var['C']['C['+str(k)+','+str(i)+']'] for i in model.N_L) for k in model.K}
    return int(max(prob, key=prob.get))

#Generate a list of predicted labels for the test set
def predicted_lab(model,X_test,var):
    label = []
    for i in range(0,len(X_test)):
        label.append(comp_label(model,list(X_test.iloc[i]),var))
    return label

#Calculate the accuracy out of sample
def accuracy(y,y_pred):
    l = [1 if y[i]==y_pred[i] else 0 for i in range(0,len(y))]
    return sum(l)/len(y)

In [28]:
var = extraction_va(model)
y_pred = predicted_lab(model,X_test,var)
yy= list(y_test)
print(accuracy(yy,y_pred))
confusion_matrix(y_test,y_pred)

0.9736842105263158


array([[10,  0,  0],
       [ 0, 16,  1],
       [ 0,  0, 11]], dtype=int64)

In [31]:
#print(var['beta'])
print(var['a'])

{'a[0,1]': -0.053154772239039985, 'a[0,2]': 1.5320302143461109e-18, 'a[0,3]': 0.053149716227350145, 'a[1,1]': -0.09791346319452933, 'a[1,2]': -2.7884646057309646e-18, 'a[1,3]': -0.09791290443470729, 'a[2,1]': 0.2510187280504541, 'a[2,2]': 5.614842712886819e-18, 'a[2,3]': 0.2510181646889559, 'a[3,1]': 0.21498217923222046, 'a[3,2]': -1.5904393511247917e-17, 'a[3,3]': 0.2149815962036867}


In [ ]:
for i in model.mu:
    print (str(model.mu[i]), model.mu[i].value)
for i in model.P:
    print (str(model.P[i]), model.P[i].value)
for i in model.p:
    print(str(model.C[i]),model.C[i].value)